In [22]:
import pyupbit
import time
from collections import deque
import datetime

# 내 집갑의 원화 잔고 조회
def get_balance_krw(balances):
    return print('내 지갑 잔고 = ', balances[0]['balance'])

# 내 지갑의 코인 잔고 조회
def get_balance_coin(balances):
    b = True
    while (b):
        if balances[1]['currency'] == 'ETH':
            balance = balances[1]['balance']
            avg_buy_price = balances[1]['avg_buy_price']
            c = (float(avg_buy_price), float(balance))
            b = False
        else:
            c = (int(0), int(0))
            b = False
    return c

def get_ticker_ma(volume, df, balances, df_2):

    ma5 = df_2['close'].rolling(window=5).mean()
    df_2.insert(len(df_2.columns), "MA5", ma5)

    ma20 = df_2['close'].rolling(window=20).mean()
    df_2.insert(len(df_2.columns), "MA20", ma20)

    ma60 = df_2['close'].rolling(window=60).mean()
    df_2.insert(len(df_2.columns), "MA60", ma60)
    
    now_price = df['close'][-1]                         # 코인의 현재가
    coin_check = get_balance_coin(balances) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0]                          # 매수 평균가
    balance = coin_check[1]                             # 코인 보유 개수
    
    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        if df_2['MA5'][-1] > df_2['MA20'][-1]:
            # 살 가격 정하기
            buy_price = volume
            buy_order(buy_price, balances)
        else:
            print('기다리는 중')
            pass
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)

        if df_2['MA5'][-1] < df_2['MA20'][-1] or avg_price > now_price:
            sell_order(balances)
        else:
            print(f"코인명: ETH, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))

# 매수 주문
def buy_order(buy_price, balances):
    while True:
        # 8000원을 빼고 
        print("내가 산 금액 = {:,.0f}원".format( buy_price))
        balances[0]['balance'] = float(balances[0]['balance']) - buy_price
        balances[1]['currency'] = 'ETH'
        # 8000원을 종가로 나누어 코인갯수 구하기
        balances[1]['balance'] = float(buy_price / df['close'][-1])
        #샀을때의 가격
        balances[1]['avg_buy_price'] = float(df['close'][-1])
        print('매수 체결')
        return

# 매도 주문
def sell_order(balances):
    while True:
        # 내가 가지고 있는 코인 갯수를 종가와 곱해서 팔았을때의 가격 측정
        sell_price = df['close'][-1] * (balances[1]['balance'])
        print("내가 판 금액 = {:,.0f}원".format(sell_price))
        balances[1]['currency'] = ''
        balances[1]['balance'] = ''
        balances[1]['avg_buy_price'] = ''
        # 팔았을 때의 가격을 기존의 지갑잔고에 더하기
        balances[0]['balance'] = balances[0]['balance'] + sell_price
        print('매도 체결')
        print('*** 내 지갑 잔고 = {:,.0f}원 ***'.format(balances[0]['balance']))
        return

i = 0
tickers = []
ma5 = []
ma20 = []
ma60 = []
balances = []
tickers = pyupbit.get_tickers(fiat = 'KRW')
# 시작 날짜 입력
day = datetime.datetime(2021,1,1)
day_2 = day
# 주식차트 기준 시간 입력
interval = 'minute60'
interval_2 = 'day'
# day = 24, minute240 = 4
time_select = 1
time_select_2 = 24
# 주식종목 입력
ticker_select = tickers[0]
# 초기 지갑 잔고 입력
volume = 10000000
# 초기 지갑설정
balances = ({'currency': 'KRW', 'balance': '{}'.format(volume), 'avg_buy_price': ''}, {'currency': '', 'balance': '', 'avg_buy_price': ''})
while True:
    i += 1
    # interval에 종목시간 입력
    df = pyupbit.get_ohlcv(ticker = ticker_select, interval = interval, to = day)
    df_2 = pyupbit.get_ohlcv(ticker = ticker_select, interval= interval_2, to = day_2)
    # 시간에 4시간씩 계속 추가
    day = day + datetime.timedelta(hours = time_select)
    if i == 24:
        i = 0
        day_2 = day_2 + datetime.timedelta(hours = time_select_2)
    print('---------------------------------------------')
    print(day)
    volume = float(balances[0]['balance'])
    # 종료 날짜 입력
    if day == datetime.datetime(2022, 1, 1):
        if balances[1]['currency'] == 'ETH':
            final_balances = float(balances[0]['balance']) + float(balances[1]['balance']) * float(df['close'][-1])
        else:
            final_balances = float(balances[0]['balance'])
        print('*** 내 지갑 잔고 = {:,.0f}원 ***'.format(final_balances))
        break
    get_ticker_ma(volume, df, balances, df_2)
    time.sleep(0.1)


---------------------------------------------
2021-01-01 01:00:00
내가 산 금액 = 10,000,000원
매수 체결
---------------------------------------------
2021-01-01 02:00:00
코인명: ETH, 수익률: 1.26%
내가산 가격 = 31612000.0, 시장가격 = 32010000.0
---------------------------------------------
2021-01-01 03:00:00
코인명: ETH, 수익률: 0.21%
내가산 가격 = 31612000.0, 시장가격 = 31679000.0
---------------------------------------------
2021-01-01 04:00:00
코인명: ETH, 수익률: 0.49%
내가산 가격 = 31612000.0, 시장가격 = 31767000.0
---------------------------------------------
2021-01-01 05:00:00
코인명: ETH, 수익률: 1.22%
내가산 가격 = 31612000.0, 시장가격 = 31999000.0
---------------------------------------------
2021-01-01 06:00:00
코인명: ETH, 수익률: 1.27%
내가산 가격 = 31612000.0, 시장가격 = 32014000.0
---------------------------------------------
2021-01-01 07:00:00
코인명: ETH, 수익률: 1.86%
내가산 가격 = 31612000.0, 시장가격 = 32200000.0
---------------------------------------------
2021-01-01 08:00:00
코인명: ETH, 수익률: 1.42%
내가산 가격 = 31612000.0, 시장가격 = 32060000.0
------------------------